In [2]:
""" Neural Network.

A 2-Hidden Layers Fully Connected Neural Network (a.k.a Multilayer Perceptron)
implementation with TensorFlow. This example is using the MNIST database
of handwritten digits (http://yann.lecun.com/exdb/mnist/).

Links:
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/

Updated by Luke Marsden at Dotscience.

Run with:

export PROJECT=<your dotscience project id>
ds run --nvidia . $PROJECT nvcr.io/nvidia/tensorflow:18.11-py2 -- \
    bash -c "pip install dotscience; python nn.py"

"""

from __future__ import print_function
import dotscience as ds; ds.interactive()
import glob

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./", one_hot=True)

ds.add_inputs(
    "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz",
    "train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz"
)

import tensorflow as tf

# Parameters
learning_rate = ds.parameter("learning_rate", 0.1)
num_steps = ds.parameter("num_steps", 100)
batch_size = ds.parameter("batch_size", 12)
display_step = 100

# Network Parameters
n_hidden_1 = ds.parameter("n_hidden_1", 256) # 1st layer number of neurons
n_hidden_2 = ds.parameter("n_hidden_2", 256) # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

saver = tf.train.Saver()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:",
        ds.summary("accuracy",
            sess.run(accuracy, feed_dict={X: mnist.test.images,
                                        Y: mnist.test.labels})))
    
    saver.save(sess, './model')

    ds.add_output("checkpoint")
    # strip "./" off the results from glob
    ds.add_outputs(*[fn[2:] for fn in glob.glob("./model.*")])

    ds.publish()


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Step 1, Minibatch Loss= 3522.7244, Training Accuracy= 0.583
Step 100, Minibatch Loss= 1988.8063, Training Accuracy= 0.833
Optimization Finished!
Testing Accuracy: 0.7009
[[DOTSCIENCE-RUN:fbed15ab-f296-4afb-9b38-5e7704822bc2]]{
    "end": "20190307T235412.926832",
    "input": [
        "t10k-labels-idx1-ubyte.gz",
        "t10k-images-idx3-ubyte.gz",
        "train-labels-idx1-ubyte.gz",
        "train-images-idx3-ubyte.gz"
    ],
    "labels": {},
    "output": [
        "model.meta",
        "model.data-00000-of-00001",
        "model.index",
        "checkpoint"
    ],
    "parameters": {
        "batch_size": "12",
        "learning_rate": "0.1",
        "n_hidden_1": "256",
        "n_hidden_2": "256",
        "num_steps": "100"
    },
    "start": "20190307T235341.994284",
    "summary": {
        "accuracy": "0.7009"
    },